In [4]:
#GOOGLE COLAB
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Freelance/TTS-Bark
from IPython.display import Image
#Image(filename='/content/drive/MyDrive/Colab Notebooks/P7-ScoringApp/P7_data_structure.png',width=800, height=500)

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Freelance/TTS-Bark


In [6]:
!apt-get install -y portaudio19-dev
!pip install pyaudio --no-build-isolation

# install bark (make sure you have torch>=2 for much faster flash-attention)
!pip install git+https://github.com/suno-ai/bark.git

!git clone https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer.git
%cd bark-voice-cloning-HuBERT-quantizer

!pip install torch torchaudio sentencepiece


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-l24t0sfp
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-l24t0sfp
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
fatal: destination path 'bark-voice-cloning-HuBERT-quantizer' already exists and is not an empty directory.
/content/drive/MyDrive/Colab Notebooks/Freelance/TTS-Bark/bark-voice-cloning-HuBERT-quantizer


In [2]:
from bark.generation import load_codec_model, generate_text_semantic
from encodec.utils import convert_audio

import torchaudio
import torch

device = 'cuda' # or 'cpu'
model = load_codec_model(use_gpu=True if device == 'cuda' else False)

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:02<00:00, 45.5MB/s]


In [7]:
# From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
from hubert.hubert_manager import HuBERTManager
hubert_manager = HuBERTManager()
hubert_manager.make_sure_hubert_installed()
hubert_manager.make_sure_tokenizer_installed()

'data/models/hubert/tokenizer.pth'

In [8]:
!pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 34.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11291806 sha256=0fbad6eb36fcb8308433b78e8f5a996849a47830879e583b5cb0b1951305594e
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=1

In [9]:
!pip install git+https://github.com/lucidrains/audiolm-pytorch.git

  Cloning https://github.com/lucidrains/audiolm-pytorch.git to /tmp/pip-req-build-vg21i46v
  Running command git clone --filter=blob:none --quiet https://github.com/lucidrains/audiolm-pytorch.git /tmp/pip-req-build-vg21i46v
  Resolved https://github.com/lucidrains/audiolm-pytorch.git to commit 986e3830a405b598f2b87796812234bccd16551f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 15.2 MB/s eta 0:00:00
  Created wheel for audiolm-pytorch: filename=audiolm_pytorch-1.5.1-py3-none-any.whl size=41520 sha256=2a55d8cd846eb93fa76552f36067e209b4448dd7e5cbbf52d7eab3cc09412311
  Stored in directory: /tmp/pip-ephem-wheel-cache-8yha1is3/wheels/0c/48/83/c37619ee7470a5724948a98f73cf2f316f670e7239caf89971
Successfully built audiolm-pytorch


In [10]:
# From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
# Load HuBERT for semantic tokens
from hubert.pre_kmeans_hubert import CustomHubert
from hubert.customtokenizer import CustomTokenizer

# Load the HuBERT model
hubert_model = CustomHubert(checkpoint_path='data/models/hubert/hubert.pt').to(device)

# Load the CustomTokenizer model
tokenizer = CustomTokenizer.load_from_checkpoint('data/models/hubert/tokenizer.pth').to(device)  # Automatically uses the right layers

In [11]:
%cd /content/drive/MyDrive/Colab Notebooks/Freelance/TTS-Bark

/content/drive/MyDrive/Colab Notebooks/Freelance/TTS-Bark


In [ ]:
ls

args.py    hubert/         prepare.py  requirements.txt
data/      LICENSE         process.py  test_hubert.py
examples/  notebook.ipynb  readme.md   ThroughRandom/


In [15]:
# Load and pre-process the audio waveform
path = "/content/drive/MyDrive/Colab Notebooks/Freelance/TTS-Bark/Constance.wav"
#path = 'macron_seg1.wav' # the audio you want to clone (under 13 seconds)
wav, sr = torchaudio.load(path)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.to(device)

In [16]:
#Load several wav files
import torchaudio

file_paths = ['audio1.wav', 'audio2.wav', 'audio3.wav']  # List of file paths

waveforms = []
sample_rate = None

for path in file_paths:
    waveform, sr = torchaudio.load(path)
    waveforms.append(waveform)

    # Check if the sample rate is consistent across all files
    if sample_rate is None:
        sample_rate = sr
    elif sr != sample_rate:
        raise ValueError("Inconsistent sample rates detected.")

# Convert the list of waveforms into a single tensor
waveforms_tensor = torch.stack(waveforms)

# Print the shape of the waveforms tensor
print("Shape of waveforms tensor:", waveforms_tensor.shape)

RuntimeError: ignored

In [ ]:
semantic_vectors = hubert_model.forward(wav, input_sample_hz=model.sample_rate)
semantic_tokens = tokenizer.get_token(semantic_vectors)

In [ ]:
# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav.unsqueeze(0))
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

In [ ]:
# move codes to cpu
codes = codes.cpu().numpy()
# move semantic tokens to cpu
semantic_tokens = semantic_tokens.cpu().numpy()

In [ ]:
import os
import numpy as np

voice_name = 'Constance'  # Set the desired name of the voice
output_dir = 'bark/assets/prompts/'
output_path = output_dir + voice_name + '.npz'

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save numpy arrays to the .npz file
np.savez(output_path, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_tokens)


In [ ]:
ls

In [ ]:
from bark.api import generate_audio
from transformers import BertTokenizer
from bark.generation import SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic

# Enter your prompt and speaker here
text_prompt = "Salut! je suis Macron, — vous faites chier. [laughs]"
voice_name = "bark/assets/prompts/Constance.npz" # use your custom voice name here if you have one

# load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [ ]:
# download and load all models
preload_models(
    text_use_gpu=True,
    text_use_small=False,
    coarse_use_gpu=True,
    coarse_use_small=False,
    fine_use_gpu=True,
    fine_use_small=False,
    codec_use_gpu=True,
    force_reload=False,
    #path="models"
)

In [ ]:
# simple generation
audio_array = generate_audio(text_prompt, history_prompt=voice_name, text_temp=0.7, waveform_temp=0.7)

In [ ]:
# generation with more control
x_semantic = generate_text_semantic(
    text_prompt,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)

x_coarse_gen = generate_coarse(
    x_semantic,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)
x_fine_gen = generate_fine(
    x_coarse_gen,
    history_prompt=voice_name,
    temp=0.5,
)
audio_array = codec_decode(x_fine_gen)

In [ ]:
from IPython.display import Audio
# play audio
Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:
from scipy.io.wavfile import write as write_wav
# save audio
filepath = "/output/audio.wav" # change this to your desired output path
write_wav(filepath, SAMPLE_RATE, audio_array)